In [1]:
import geopandas as gpd
import fiona 
import pandas as pd
from bs4 import BeautifulSoup
import shapely
import re
from shapely.geometry import Polygon

In [2]:
def extract_border(df):
    border = df.copy()
    border['new_column'] = 0
    border = border.dissolve(by='new_column')
    border = border[['geometry']]
    border['geometry'] = border['geometry'].buffer(0.000001)
    border.index.names = ['ID']
    
    return border

def parse_html(row, dic='None'):
    html_str = row['Description']
    parsed_html = BeautifulSoup(html_str)
    array = parsed_html.body.find_all('td')
    array = array[2:]
    array = [clean_html(str(tag)) for tag in array]
    dic = dict(array[i:i+2] for i in range(0, len(array), 2))
    if not(dic['FID']):
        print(dic)
    return pd.Series(dic)
    
def clean_html(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [3]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
quadras_df = gpd.read_file('./data/BASES/quadras/quadras.kml')

# Drop Z dimension of polygons that occurs often in kml 
quadras_df.geometry = quadras_df.geometry.map(lambda polygon: shapely.ops.transform(lambda x, y, z: (x, y), polygon))

In [4]:
quadras_df = quadras_df.join(quadras_df.apply(parse_html, axis=1))
quadras_df = quadras_df.drop('Description', 1)
quadras_df = quadras_df[['FID', 'SETOR', 'geometry']]

In [ ]:
lotes_geral = gpd.read_file('./data/BASES/lotes_geral/Lotes.kml')



In [ ]:
lotes_geral.plot()

In [5]:
setores = gpd.read_file('./data/shp/jp_shp.shp')
setores = setores[['CD_GEOCODI', 'NM_BAIRRO', 'geometry']]

In [6]:
bairros = setores.dissolve(by='NM_BAIRRO', aggfunc='sum')
bairros['geometry'] = bairros['geometry'].buffer(0.0000001)

In [7]:
bairros_interesse_menor = ['Torre', 'Tambauzinho', 'Expedicionários', 'Miramar']
bairros_interesse_maior = ['Torre', 'Tambauzinho', 'Expedicionários', 'Miramar', 'Pedro Gondim', 'Estados']

setores_recorte_maior = setores.loc[setores['NM_BAIRRO'].isin(bairros_interesse_maior)]
setores_recorte_menor = setores.loc[setores['NM_BAIRRO'].isin(bairros_interesse_menor)]

In [8]:
limite_recorte_maior = extract_border(setores_recorte_maior)
limite_recorte_menor = extract_border(setores_recorte_menor)
limite_municipio = extract_border(bairros)
limite_municipio['geometry'][0] = Polygon(limite_municipio['geometry'][0].exterior)

In [9]:
setores = setores.to_crs(epsg=3857)
bairros = bairros.to_crs(epsg=3857)
limite_recorte_maior = limite_recorte_maior.to_crs(epsg=3857)
limite_recorte_menor = limite_recorte_menor.to_crs(epsg=3857)
limite_municipio = limite_municipio.to_crs(epsg=3857)
quadras_df = quadras_df.to_crs(epsg=3857)


setores.to_file("./data/BASES/setores/setores.shp")
bairros.to_file("./data/BASES/bairros/bairros.shp")
limite_recorte_maior.to_file("./data/BASES/limite_recorte/limite_recorte_maior.shp")
limite_recorte_menor.to_file("./data/BASES/limite_recorte/limite_recorte_menor.shp")
limite_municipio.to_file("./data/BASES/limite_municipio/limite_municipio.shp")
quadras_df.to_file("./data/BASES/quadras/quadras.shp")

In [10]:
#bairros['NM_BAIRRO'] = bairros.index

In [11]:
bairros_interesse_maior = ['Torre', 'Tambauzinho', 'Expedicionários', 'Miramar', 'Pedro Gondim', 'Estados']
bairros_interesse_menor = ['Torre', 'Tambauzinho', 'Expedicionários', 'Miramar']
bairros_recorte_maior = bairros.loc[bairros.index.isin(bairros_interesse_maior)]

In [12]:
bairros_recorte_maior = bairros_recorte_maior.reindex(['Torre', 'Expedicionários', 'Tambauzinho', 'Miramar', 'Pedro Gondim', 'Estados'])

In [13]:
bairros_recorte_maior['coordsX'] = bairros_recorte_maior['geometry'].apply(lambda x: x.centroid.coords[:][0][0])
bairros_recorte_maior['coordsY'] = bairros_recorte_maior['geometry'].apply(lambda x: x.centroid.coords[:][0][1])
bairros_recorte_maior['geometry'] = bairros_recorte_maior['geometry'].buffer(0.001)


bairros_recorte_menor = bairros_recorte_maior.loc[bairros_recorte_maior.index.isin(bairros_interesse_menor)]

bairros_recorte_maior = bairros_recorte_maior.to_crs(epsg=3857)
bairros_recorte_menor = bairros_recorte_menor.to_crs(epsg=3857)
bairros_recorte_menor.to_file("./data/BASES/bairros/bairros_recorte_menor.shp")
bairros_recorte_maior.to_file("./data/BASES/bairros/bairros_recorte_maior.shp")